# Datathon Team 13

9/20/2021

**Problem definition:** Want to predict turnaround time

In [ ]:
!pip install --upgrade pandas
!pip install geopy
!pip install xgboost
!pip install holidays

<div class="alert alert-block alert-info">

### Load packages
</div>

In [ ]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import date, datetime
import random
from geopy.distance import distance
import xgboost as xgb
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
tqdm.pandas()
import holidays
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from math import sqrt

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)
outputs = []

def clean_data(thisDf): 
    thisDf['ORDER_CODE'] = thisDf['ORDER_CODE'].astype(str)
    # remove anything with a negative TAT
#     thisDf = thisDf[(thisDf['TAT_HOUR']>=0) | (thisDf['TAT_HOUR'].isnull())]
#     # remove anything that is over the 99th percentile
#     thisDf = thisDf[(thisDf['TAT_HOUR']<=thisDf['TAT_HOUR'].quantile(.99)) | (thisDf['TAT_HOUR'].isnull())]
    return thisDf
    
def initial_drop_columns(thisDf): 
    return thisDf.drop([
        #'RECORD_ID',
        'PERFORMING_LAB_NAME',
        'BU_NAME',
        #'BU_LATITUDE',
        #'BU_LONGITUDE',
        'ORDERING_LAB_CODE',
        'ORDERING_LAB_NAME',     
        'BILLING_LEGAL_ENTITY',
        'ACCOUNT_NUMBER',
        'ACCOUNT_NAME',
        'ACCOUNT_STATE',
        'ACCOUNT_ZIP_CODE',
        'SPECIALTY_DESC',
        'PHYSICIAN_NPI',
        'PHYSICIAN_NAME',
        'BILL_ONLY_INDICATOR',
        'ORDER_UNIT_CODE',#come back to this 
        'ORDER_NAME',
        'ORDER_CODE_MNEMONIC',
        'PUBLISHED_TAT',
        'MAX_TAT',
        'STAT_ROUTINE_INDICATOR'   
    ], axis=1)



def get_distances(thisDf):

    def geo_distance(x):

        try: 
            return distance( (x['PERFORMING_LAB_LATITUDE'], x['PERFORMING_LAB_LONGITUDE']),
                               (x['ORDERING_LAB_LATITUDE'], x['ORDERING_LAB_LONGITUDE'])
                           ).miles
        except:
            return 0
        
    try:
        thisDf['Distance'] = thisDf.progress_apply(geo_distance, axis=1)
    except:
        thisDf['Distance'] = thisDf.apply(geo_distance, axis=1)
    return thisDf

def do_concats(thisDf): 
    # df_t['Lab_Order'] = df_t['LAB_SYSTEM_ID'].astype(str) + df_t['ORDER_CODE'].astype(str)
    # df_t['Performing_Lab'] = df_t['PERFORMING_LAB_SITE_TYPE'].astype(str) + df_t['PERFORMING_LAB_CODE'].astype(str)

    return thisDf

def update_add_on_exists(thisDf):
    thisDf['Add_On_Exists'] =thisDf['ADD_ON_ORDER_DATE'].isnull()
    thisDf['Add_On_Exists'] =thisDf['Add_On_Exists'].apply(lambda x: 0 if x is True else 1)
    return thisDf

def do_date_stuff(thisDf):
#     def day_of_week(x):
#         return x.day_name()

    thisDf['COLLECTION_DATE'] = pd.to_datetime(thisDf['COLLECTION_DATE'])
    thisDf['ACCESSION_DATE'] = pd.to_datetime(thisDf['ACCESSION_DATE'])
    thisDf['Collection_DOW'] = thisDf['COLLECTION_DATE'].dt.day_name()
    thisDf['Accession_DOW'] = thisDf['ACCESSION_DATE'].dt.day_name()
    
    #check to see if holiday
    us_holidays = holidays.US()
    
    thisDf['Accession_is_Holiday'] = thisDf['ACCESSION_DATE'].apply(lambda x: x in us_holidays)
    thisDf['Collection_is_Holiday'] = thisDf['COLLECTION_DATE'].apply(lambda x: x in us_holidays)
    thisDf['Collection_is_Holiday'] = thisDf['Collection_is_Holiday'].apply(lambda x: 1 if x is True else 0)
    thisDf['Accession_is_Holiday'] = thisDf['Accession_is_Holiday'].apply(lambda x: 1 if x is True else 0)
    
    
    # get collection hour
    thisDf['Collection_Hour'] = thisDf['COLLECTION_DATE'].dt.hour
    
    # do hours between collection/accession
    thisDf['Hours_Collection_to_Accession'] = thisDf['ACCESSION_DATE'] - thisDf['COLLECTION_DATE']
    thisDf['Hours_Collection_to_Accession'] = thisDf['Hours_Collection_to_Accession'].dt.total_seconds()/60/60
    
    thisDf['Bad_Accession_Date'] = thisDf['COLLECTION_DATE'] > thisDf['ACCESSION_DATE'] 
    thisDf['Bad_Accession_Date'] = thisDf['Bad_Accession_Date'].apply(lambda x: 1 if x is True else 0)

    
    return thisDf

def second_drop_columns(thisDf):
    return thisDf.drop(
        [
            'PERFORMING_LAB_LATITUDE',
            'PERFORMING_LAB_LONGITUDE',
            'ORDERING_LAB_LATITUDE',
            'ORDERING_LAB_LONGITUDE',
            'BU_LATITUDE',
            'BU_LONGITUDE',
            'COLLECTION_DATE',
            'ACCESSION_DATE',
            'ADD_ON_ORDER_DATE',
            'WORKLIST_CODE',
            'PERFORMING_LAB_SITE_TYPE',
            #'ORDER_CODE',
            #'PERFORMING_LAB_CODE',
        ], axis=1)

def get_dummy_vars(thisDf):
    return pd.get_dummies(thisDf, drop_first = True)

<div class="alert alert-block alert-info">

### Load data
</div>

In [ ]:
%%time 

s3_bucket = 'dgx-datathon-data/validation'
filename = 'datathon_validation.tab'
data_location = 's3://{}/{}'.format(s3_bucket, filename)

df_validation_ = pd.read_csv(data_location, sep='\t')

In [ ]:
%%time 

s3_bucket = 'dgx-datathon-data/full'
filename = 'datathon.tab'
data_location = 's3://{}/{}'.format(s3_bucket, filename)

df_ = pd.read_csv(data_location, sep='\t')
df_.shape

In [ ]:
# df = df.sample(n=1000000, random_state=42)
# df.shape
# df.to_csv('df_sample_1m.csv', index=False)
# df.to_csv('s3://dgx-team13-s3/nasser-autopilot/df_sample_1m.csv', index=False)

In [ ]:
print(df_validation_.shape)
print(df_.shape)

### Concatenate validation and regular

In [ ]:
%%time 

df_validation = df_validation_.copy()
df = df_.copy()
df['Source'] = 'Full'
df_validation['Source'] = 'Evaluation'

In [ ]:
df = pd.concat([df_validation, df])

<div class="alert alert-block alert-warning">

## Checkpoint 0
</div>

<div class="alert alert-block alert-info">

### Initial cleaning of data
</div>

In [ ]:
# initial data column drops
df = initial_drop_columns(df)

<div class="alert alert-block alert-info">

### Transformations on the data
</div>

* number of times specimen is collected, as well as actual specimen collection time
    * also look for morning/afternoon/evening
* find when BU region is different from performing (binary)

<div class="alert alert-block alert-warning">

## Checkpoint 1
</div>

In [ ]:
df_t = df.copy()

In [ ]:
df_t = clean_data(df_t)

In [ ]:
df_t = do_concats(df_t)

In [ ]:
%%time

df_lat_lon = df_t[['PERFORMING_LAB_LATITUDE', 'PERFORMING_LAB_LONGITUDE', 'ORDERING_LAB_LATITUDE', 'ORDERING_LAB_LONGITUDE']]
df_lat_lon.drop_duplicates(inplace=True)
df_lat_lon = get_distances(df_lat_lon)

df_t = pd.merge(
    df_t,
    df_lat_lon,
    on=['PERFORMING_LAB_LATITUDE', 'PERFORMING_LAB_LONGITUDE', 'ORDERING_LAB_LATITUDE', 'ORDERING_LAB_LONGITUDE'],
    how='left'
)

In [ ]:
df_t = update_add_on_exists(df_t)

In [ ]:
%%time 

df_t = do_date_stuff(df_t)
# morning/afternoon/evening transformation

<div class="alert alert-block alert-warning">

## Checkpoint !!
</div>

In [ ]:
#CHECKPOINT
df_checkpoint = df_t.copy()

In [ ]:
print(df_t.shape)
df_t['Source'].value_counts()

In [ ]:
# initial data column drops
df_t = second_drop_columns(df_t)

In [ ]:
# Get an idea of how many new features we'll get from dummy explosion
for col in list(df_t):
    if (df_t[col].dtype =='object'):
        print('col:', col, 'unique vals: ', df_t[col].nunique() )

In [ ]:
#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (15,6))
sns.set(font_scale=1.0)
sns.boxplot(x="MARKET_SEGMENT_DESC", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'MARKET_SEGMENT_DESC']].sample(200000))

In [ ]:
#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (8,6))
sns.set(font_scale=1.0)
sns.boxplot(x="Collection_Hour", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'Collection_Hour']].sample(200000))

In [ ]:
#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (13,6))
sns.set(font_scale=1.0)
sns.boxplot(x="PERFORMING_REGION", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'PERFORMING_REGION']].sample(200000))

In [ ]:


#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (3,6))
sns.set(font_scale=1.0)
sns.boxplot(x="Collection_is_Holiday", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'Collection_is_Holiday']].sample(200000))


In [ ]:


#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (8,6))
sns.set(font_scale=1.0)
sns.boxplot(x="Collection_DOW", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'Collection_DOW']].sample(200000))


In [ ]:

#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (12,6))
sns.set(font_scale=1.0)
sns.boxplot(x="LAB_SYSTEM_ID", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'LAB_SYSTEM_ID']].sample(200000))


In [ ]:
#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (12,6))
sns.set(font_scale=1.0)
sns.boxplot(x="ORDERING_LAB_SITE_TYPE", y="TAT_HOUR", data=df_t[['TAT_HOUR',  'ORDERING_LAB_SITE_TYPE']].sample(200000))


In [ ]:
plt.figure(figsize = (12,12))
sns.set(font_scale=1.1)
sns.pairplot(df_t[['TAT_HOUR', 'Distance', 'Collection_Hour', 'Hours_Collection_to_Accession', 'PERFORMING_REGION']].sample(10000))

In [ ]:
plt.figure(figsize = (8,2))
sns.set(font_scale=1.1)

sns.heatmap(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR', 'Distance', 'Collection_Hour', 'Hours_Collection_to_Accession']].sample(2000).corr(), annot=True)

<div class="alert alert-block alert-warning">

## Checkpoint !!
</div>

In [ ]:
df_checkpoint_2 = df_t.copy()

In [ ]:
%%time
df_t = get_dummy_vars(df_t)

<div class="alert alert-block alert-info">

### Building the model
</div>


In [ ]:
%%time 

# separate labeled data
df_labeled = df_t[df_t['Source_Full']==1].copy()
df_labeled.drop(['Source_Full'], axis=1, inplace=True)

# of labeled data, get a 3M samples that will be used for train/test
df_model = df_labeled.sample(n=3000000, random_state=42)

# get the data that will not be used for train/test for validation
training_test_ids = list(df_model['RECORD_ID'])
df_validation = df_labeled[~df_labeled['RECORD_ID'].isin(training_test_ids)]

# this is what we wil be evaluated on
df_model_eval = df_t[df_t['Source_Full']==0].copy()
df_model_eval.drop(['Source_Full'], axis=1, inplace=True)

# drop record IDs
df_model.drop(['RECORD_ID'], axis=1, inplace=True)
df_model_eval.drop(['RECORD_ID'], axis=1, inplace=True)

print(df_labeled.shape)
print(df_model.shape)
print(df_validation.shape)
print(df_model_eval.shape)

In [ ]:
%%time 

# get data in array format because that's what ML models prefer
X = np.array(df_model.drop(['TAT_HOUR'], axis=1)).astype('float32')
y = np.array(df_model['TAT_HOUR']).astype('float32')


In [ ]:
%%time 

y = y.reshape(-1,1)

In [ ]:
%%time 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [ ]:
X_test.shape

In [ ]:
model

In [ ]:
%%time 

model = xgb.XGBRegressor(
    objective='reg:squarederror',
    learning_rate=None,
    max_depth=None,
#     reg_lambda=0.04328041793594834,
#     min_child_weight=22.356147665843558,
#     gamma=33.61489625428083,
    n_estimators=100    
)

model.fit(X_train, y_train)

In [ ]:
%%time 
model_score = model.score(X_test, y_test)
print(model_score)

y_predict = model.predict(X_test)
k = X_test.shape[1]
n = len(X_test)
RMSE = float(format(np.sqrt(mean_squared_error(y_test, y_predict)),'.3f'))
MSE = mean_squared_error(y_test, y_predict)
MAE = mean_absolute_error(y_test, y_predict)
r2 = r2_score(y_test, y_predict)
adj_r2 = 1-(1-r2)*(n-1)/(n-k-1)

print('RMSE =',RMSE, '\nMSE =',MSE, '\nMAE =',MAE, '\nR2 =', r2, '\nAdjusted R2 =', adj_r2) 

In [ ]:
model_score = model.score(X_test, y_test)
notes = 'BASELINE'
now = datetime.now() # current date and time
time = now.strftime("%m_%d_%Y__%H_%M_%S")

outputs.append([    
    time,
    model.__dict__['n_estimators'],
    model.__dict__['max_depth'],
    model.__dict__['learning_rate'],
    model_score,
    X.shape,
    notes]
)


df_outputs = pd.read_csv('outputs.csv')
df_outputs = pd.concat([   
    df_outputs,
    pd.DataFrame(outputs, columns=['time','n_estimators', 'max_depth', 'learning_rate', 'accuracy', 'X shape', 'notes']),
])
df_outputs.drop_duplicates(inplace=True)
df_outputs.to_csv('outputs.csv', index=False)
df_outputs

### Validation model

In [ ]:
for col in list(df_validation_1m):
    if col not in list(df_model):
        print(col)


In [ ]:
%%time 

df_validation_1m = df_validation.head(1000000)

X_validation = np.array(df_validation_1m.drop(['RECORD_ID','TAT_HOUR'], axis=1)).astype('float32')
y_validation = np.array(df_validation_1m['TAT_HOUR']).astype('float32')

In [ ]:
y_hat_validation

In [ ]:
%%time 
y_hat_validation = model.predict(X_validation)
r2_score(y_hat_validation.astype(int), y_validation)

In [ ]:
df_validation.shape

In [ ]:
df_validation_1m['y_hat']=y_hat_validation

In [ ]:
df_validation_1m[['y_hat','TAT_HOUR']]

### Evaluation model

In [ ]:
%%time 

X_evaluation = np.array(df_model_eval.drop(['TAT_HOUR'], axis=1)).astype('float32')

In [ ]:
X_evaluation.shape

In [ ]:
y_hat_evaluation = model.predict(X_evaluation)

In [ ]:
df_model_eval = df_t[df_t['Source_Full']==0].copy()


In [ ]:
df_model_eval['TAT_HOUR']=y_hat_evaluation.astype(int)

In [ ]:
df_model_eval.head()

In [ ]:
df_final = pd.merge(
    df_validation_,
    df_model_eval[['RECORD_ID','TAT_HOUR']],
    on='RECORD_ID',
    how='left'
)

#df_final.to_csv('s3://dgx-team13-s3/nasser-autopilot/df_evaluation.csv', index=False)

In [ ]:
df_SUBMIT = df_model_eval[['RECORD_ID','TAT_HOUR']]

In [ ]:
df_SUBMIT.head()

In [ ]:
df_prev = pd.read_csv('s3://dgx-team13-s3/nasser-autopilot/df_evaluation.csv')

In [ ]:
#df_SUBMIT.head()
df_SUBMIT.to_csv('s3://dgx-team13-s3/nasser-autopilot/Team13_prediction.csv', index=False)

### Pass validation data through model

In [ ]:
df_SUBMIT.head()

In [ ]:
#df_SUBMIT.head()
df_SUBMIT.to_csv('s3://dgx-team13-s3/submission/Team13_prediction.csv', index=False)

In [ ]:

#sns.boxplot(df_t[df_t['TAT_HOUR']<=df_t['TAT_HOUR'].quantile(.95)][['TAT_HOUR',  'PERFORMING_REGION']].sample(2000))
plt.figure(figsize = (12,6))
sns.set(font_scale=1.0)
sns.boxplot(x="LAB_SYSTEM_ID", y="TAT_HOUR", data=df_final[['TAT_HOUR',  'LAB_SYSTEM_ID']].sample(200000))


In [ ]:
df_final.drop(['TAT_HOUR_x'], inplace=True, axis=1)
df_final['TAT_HOUR'] = df_final['TAT_HOUR_y']